<a href="https://colab.research.google.com/github/sclaudiobr/geocloud/blob/main/GeoCloud02_Download_de_dados_Tematicos_do_IBGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GeoCloud 02 - Download de dados Temáticos do IBGE

Ferramenta para recortar  e baixar dados temáticos disponíveis na base de dados do IBGE, através da biblioteca [WFs](https://inde.gov.br/) selecionados a partir dos limites municipais da base de dados do IBGE através da biblioteca [geoBR](https://pypi.org/project/geobr/). Neste  algoritmo foram selecionados os temas Geologia, Geomorfologia, Solos e Vegetação. Ao final do processo uma pasta compactada é gerada para o download de cada camada de dados.

<hr>
@ autor Claudio Roberto Oliveira da Silva 
<p> Bel. Lic. Ciências Biológicas - UNL/AM - 2005
<p>MSc. Modelagem Computacional, Eng. Civil - COPPE/UFRJ - 2008
<p>Tecnólogo de Geoprocessamento, Cursando - UFSM - 2018
<p>email: <a href= "mailto:claumao@gmail.com"> claumao@gmail.com</a>
<p> Repositório: <a href="https://github.com/sclaudiobr/"> GitHub</a>

In [ ]:
# Baixa e instala bibliotecas
!pip install geobr # Base de dados do IBGE, malhas municipais, biomas, microrregiões, sedes
!pip install shapely  # Conversor e leitor de arquivos shp 
!pip install OWSLib # Acesso ao banco de dados WFS/WMS/OWS
!pip install pygeos # Biblioteca de apoio ao Geopandas
!pip install --upgrade shapely
!pip install rtree

In [ ]:
from genericpath import exists
#------ Importa bibliotecas --------
import numpy as np
import geobr as br
import geopandas as gpd
import os
import owslib.wfs as ows
from owslib.wfs import WebFeatureService
from shapely.geometry import box
import zipfile
# --- cria diretório usando o shell
!mkdir shp/
### ---- FUNÇÃO BAIXAR DADOS -----------------------------
### Esta função recebe os parâmetros UF, nome no municíípio e identificador da camada de dados de interesse. '''
def gera_dados(uf,mun,id):
   df = br.read_municipality(code_muni=uf)
   df = df[df.name_muni==mun]
   dfbox = box(*df.total_bounds).bounds
   latitude,longitude = np.array(dfbox)[1],np.array(dfbox)[2]
   wfs_url = 'https://geoservicos.ibge.gov.br/geoserver/wfs'
   # Conectando ao serviço do GeoServer WFS.
   wfs = WebFeatureService(wfs_url, version='2.0.0')
   data = wfs.getfeature(typename=id, bbox=dfbox, outputFormat='JSON')
   fn = 'df.geojson'
   with open(fn, 'wb') as fh:
      fh.write(data.read())
   layers = gpd.read_file(fn)
   recorte = gpd.clip(layers, df,keep_geom_type=False,)
   recorte.to_file('shp/df.shp', driver='ESRI Shapefile')

# ----- FUNÇÃO COMPACTA PASTAS MANTENDO A ESTRUTURA
def zip(src, dst):
    zf = zipfile.ZipFile("%s.zip" % (dst), "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            print('zipping %s as %s' % (os.path.join(dirname, filename), arcname))
            zf.write(absname, arcname)
    zf.close()

# Identficadores das camadas
geo = 'CREN:Geologia_area_Brasil'
geom = 'CREN:Geomorfologia_area_Brasil'
ped = 'CREN:Pedologia_area_Brasil'
veg = 'CREN:vegetacao_area_brasil'


#### TROCA OS DADOS AQUI ########################
UF = 'AM'                       # <---- Estado aqui
mun = "Manaus"                  #<--- Municipio Aqui (nome com acentuação)
camada = geom                   # <-- Camada de dados id (veg,geo,geom,ped)
nome_do_zip = camada[5:-12]      # <-  (nome sem acento)
#############################################
gera_dados(UF,mun,camada)

zip("shp",nome_do_zip)

# Plotando o mapa
gdf =gpd.read_file('shp/df.shp')
gdf.plot(figsize=(40,10), alpha=0.5, edgecolor='k', cmap="jet",column='legenda', legend=True)